# Webscrapeo : Ubica tu casilla

### Paquetes

In [130]:
import json
import time
import requests
import numpy as np
import pandas as pd

# Local imports
from keys import proxies

requests.adapters.DEFAULT_RETRIES = 5 # increase retries number

### Pipeline

In [92]:
API = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo-with-cmr-uts?e=1&s=488"

headers = {"authority": "us-central1-ine-ubica-tu-casilla.cloudfunctions.net",
            "method": "GET",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br",
            "origin": "https://ubicatucasilla.ine.mx",
            "referer": "https://ubicatucasilla.ine.mx/",
            "accept-language": "en,es-419;q=0.9,es;q=0.8",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36",
            "x-utc-service-id": "ubicatucasilla",
            "x-utc-version-id": "2022",
            "sec-ch-ua": 'Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "Windows",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "cross-site",
}

In [93]:
def get_raw_df(API, num_edo, num_casilla, session):
    """
    Extrae el dataframe de una casilla a partir del API
     
    Parameters
    API: string
        URL del API
    num_edo: int
        Numero de estado
    num_casilla: int
        Numero de casilla a extraer
    """
    path = "/utc-api/search/rasgo-with-cmr-uts?e={num_edo}&s={num_casilla}"
    headers["path"] = path 

    new_API = API.replace("e=1", f"e={num_edo}")
    new_API = new_API.replace("s=488", f"s={num_casilla}")
    success = 0
    while success == 0:
        try:
            r = session.get(new_API, headers=headers, timeout=5)
            success = 1
        except Exception as err:
            time.sleep(1.5)
            print(f"Retrying request: {err}")

    if r.json()["status"] == False:
        raw_df = pd.DataFrame([])
        if 'cmr' in r.json().keys():
            content = r.json()

        else:
            raw_df = pd.DataFrame([])
            content = {}
    
    else:
        content = r.json()['data'][0]['items']
        raw_df = pd.DataFrame(content[0]['apellidos'])
        raw_df['domicilio'] = content[0]['data']['domicilio']
        raw_df['ubicacion'] = content[0]['data']['ubicacion']

        raw_df['entidad'] = content[0]['entidadSede']
        raw_df['distitoSede'] = content[0]['distritoSede']
        raw_df['distritoLocal'] = content[0]['distrito_localSede']
        raw_df['latitud'] = content[0]['reference']['dce']['geom']['coordinates'][0]
        raw_df['longitud'] = content[0]['reference']['dce']['geom']['coordinates'][1]
        
        raw_df 

    return raw_df, content

In [94]:
def get_clean_df(raw_df, content):
    """
    Saca clean_df a partir del raw_df y pone varios 
    dfs de acuerdo al numero de secciones
    
    Parameters
    ----------
    raw_df: dataframe
        Dataframe de la casilla
    content: dict
        Diccionario con la informacion de la casilla
    
    """
    raw_secciones = content[0]['secciones']

    clean_secciones = [int(x) for x in raw_secciones]
    complete_df = pd.DataFrame([])

    for clean_seccion in clean_secciones:
        copy_df = raw_df.copy()
        copy_df['seccion'] = clean_seccion

        complete_df = pd.concat([complete_df, copy_df], 
                                ignore_index=True)

    return complete_df, clean_secciones

In [95]:
def extract_estado(API, num_edo, session):
    """
    Extrae todas las casillas de un estado

    Parameters
    ----------
    API : string
        API de INE
    num_edo : int
        Numero de estado a extraer

    Returns
    -------
    estado_df : dataframe
        Dataframe con todas las casillas de un estado
    pending_sections : list
        Lista con todas las secciones pendientes
    """

    all_secciones = np.arange(1, 7000)
    extracted_sections = []
    estado_df = pd.DataFrame([])
    pending_sections = []
    counter = 0
    non_existent_counter = 0

    for seccion in all_secciones:
        counter += 1
        if counter % 100 == 0:
            print(f"Extracted {counter} casillas")
            time.sleep(20)

        # Si la seccion ya la extrajimos
        if seccion in extracted_sections:
            continue

        raw_df, content = get_raw_df(API, num_edo, seccion, session)

        if content:

            if not isinstance(content, list):
                time.sleep(3)
                pending_sections.append(seccion)
                # Si falla el API, continuamos (si existe la casilla, pero pide datos del INE)
                continue
            else:
                # Sacar datos limpios y quitar casillas que ya hayamos extraido
                complete_df, clean_secciones = get_clean_df(raw_df, content)
                estado_df = pd.concat([estado_df, complete_df], ignore_index=True)
                # Agregar las secciones ya extraidas
                extracted_sections.extend(clean_secciones)

        # Si el content está vacío, continuamos
        else:
            non_existent_counter += 1
            
            if non_existent_counter >= 30:
                print("40 casillas no encontradas, stopping...")
                break
            else:
                continue
    
    return estado_df, pending_sections

In [122]:
def get_casillas_especiales():
    """
    Extrae casillas especiales de la API

    """
    API_especial = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo?e=31&special=true"
    specials_df = pd.DataFrame([])

    with requests.Session() as session:
        for estado in np.arange(1, 33):

            new_API = API_especial.replace("e=31", f"e={estado}")
            r = session.get(new_API, headers=headers)
            content = r.json()['data']

            estado_df = pd.DataFrame([])
            for sub_content in content:
                raw_df = pd.DataFrame(sub_content['rasgo']).reset_index()
                raw_df = raw_df.iloc[1:,]
                raw_df.reset_index(drop=True, inplace=True)

                estado_df = pd.concat([estado_df, raw_df], ignore_index=True)
            
            estado_df['entidad'] = estado

            specials_df = pd.concat([specials_df, estado_df], 
                                    ignore_index=True)
            
            time.sleep(1)
        
        requests.post(new_API, headers={"Connection": "close"})

    return specials_df

### Pipeline - Todas las casillas

In [125]:
estados = np.arange(1, 32)
mexico_df = pd.DataFrame([])
pendientes_dic = {}

session = requests.Session()

for estado in estados:    
    estado_df, pending_sections = extract_estado(API, estado, session)
    pendientes_dic[estado] = pending_sections

    mexico_df = pd.concat([mexico_df, estado_df], ignore_index=True)

    print(f"========== Extracted {estado} ===========")
    time.sleep(25)

requests.post(API, headers={"Connection": "close"})


Extracted 100 casillas
Extracted 200 casillas
Extracted 300 casillas
Extracted 400 casillas
Extracted 500 casillas
Extracted 600 casillas
40 casillas no encontradas, stopping...
========== Extracted 1 ===========
Extracted 100 casillas
Extracted 200 casillas
Extracted 300 casillas
Extracted 400 casillas
Extracted 500 casillas
Extracted 600 casillas
Extracted 700 casillas
Extracted 800 casillas
Extracted 900 casillas
Extracted 1000 casillas
Extracted 1100 casillas
Extracted 1200 casillas
Extracted 1300 casillas
Extracted 1400 casillas
Extracted 1500 casillas
Extracted 1600 casillas
Extracted 1700 casillas
Extracted 1800 casillas
Extracted 1900 casillas
Extracted 2000 casillas
40 casillas no encontradas, stopping...
========== Extracted 2 ===========
Extracted 100 casillas
Extracted 200 casillas
Extracted 300 casillas
Extracted 400 casillas
Extracted 500 casillas
40 casillas no encontradas, stopping...
========== Extracted 3 ===========
Extracted 100 casillas
Extracted 200 casillas
Extra

<Response [200]>

In [126]:
mexico_df

,casilla,inicial,final,domicilio,ubicacion,entidad,distitoSede,distritoLocal,latitud,longitud,seccion
0,B,ABOYTES DOÑAS,LLAMAS DELGADO,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,-102.321935,21.929537,1
1,C1,LLAMAS ESPERON,ZUÑIGA GONZALEZ,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,-102.321935,21.929537,1
2,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,-102.311723,21.909189,2
3,C1,LOERA CHAVEZ,ZURITA ALVARADO,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,-102.311723,21.909189,2
4,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,-102.311723,21.909189,4
...,...,...,...,...,...,...,...,...,...,...,...
151490,C1,DIAZ SANTOS,PECH CAAMAL,"CALLE 17 POR 22 B Y 10 A, SIN NÚMERO, COLONIA ...",ESCUELA SECUNDARIA TÉCNICA NÚMERO 2 RAQUEL DZI...,31,5,8,-89.737702,20.889688,1132
151491,C2,PECH CAAMAL,ZUÑIGA ISLAS,"CALLE 17 POR 22 B Y 10 A, SIN NÚMERO, COLONIA ...",ESCUELA SECUNDARIA TÉCNICA NÚMERO 2 RAQUEL DZI...,31,5,8,-89.737702,20.889688,1132
151492,B,ABAN ABAN,DIAZ SANTOS,"CALLE 17 POR 22 B Y 10 A, SIN NÚMERO, COLONIA ...",ESCUELA SECUNDARIA TÉCNICA NÚMERO 2 RAQUEL DZI...,31,5,8,-89.737702,20.889688,991
151493,C1,DIAZ SANTOS,PECH CAAMAL,"CALLE 17 POR 22 B Y 10 A, SIN NÚMERO, COLONIA ...",ESCUELA SECUNDARIA TÉCNICA NÚMERO 2 RAQUEL DZI...,31,5,8,-89.737702,20.889688,991


In [127]:
mexico_df.to_csv("casillas_edos-1-32.csv", encoding="utf-8-sig", 
                    index=False)        

In [128]:
pendientes_dic

{1: [81, 318, 325, 368, 401, 459],
 2: [13,
  108,
  123,
  126,
  134,
  166,
  170,
  185,
  200,
  506,
  568,
  571,
  661,
  662,
  692,
  710,
  721,
  723,
  725,
  830,
  1152,
  1189,
  1196,
  1226,
  1230,
  1278,
  1279,
  1281,
  1282,
  1293,
  1294,
  1356,
  1368,
  1421,
  1485,
  1520,
  1800],
 3: [64, 83, 252, 253, 275, 285, 337],
 4: [229, 273, 278, 331, 332, 339, 340, 342, 413, 437],
 5: [9,
  57,
  60,
  62,
  65,
  101,
  103,
  107,
  118,
  219,
  224,
  253,
  443,
  519,
  533,
  538,
  638,
  640,
  641,
  653,
  668,
  672,
  675,
  678,
  717,
  830,
  992,
  1018,
  1037,
  1169,
  1260,
  1314,
  1469,
  1713],
 6: [198],
 7: [1,
  14,
  19,
  160,
  261,
  358,
  393,
  400,
  402,
  415,
  416,
  462,
  471,
  478,
  481,
  482,
  486,
  491,
  501,
  507],
 8: [29,
  42,
  44,
  58,
  59,
  77,
  93,
  230,
  246,
  247,
  255,
  357,
  366,
  372,
  387,
  610,
  611,
  613,
  670,
  879,
  881,
  882,
  883,
  895,
  896,
  911,
  920,
  972,
  102

### Casillas pendientes donde piden datos del Elector

In [129]:
API = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/electorUts?elector=DMMLFD97071209H300&entidad=9&flag=7&seccion=767&service=ubicatucasilla"

In [142]:
clean_pendientes = {}

for estado in pendientes_dic.keys():
    clean_pendientes[str(estado)] = str(pendientes_dic[estado])

In [143]:
clean_pendientes

{'1': '[81, 318, 325, 368, 401, 459]',
 '2': '[13, 108, 123, 126, 134, 166, 170, 185, 200, 506, 568, 571, 661, 662, 692, 710, 721, 723, 725, 830, 1152, 1189, 1196, 1226, 1230, 1278, 1279, 1281, 1282, 1293, 1294, 1356, 1368, 1421, 1485, 1520, 1800]',
 '3': '[64, 83, 252, 253, 275, 285, 337]',
 '4': '[229, 273, 278, 331, 332, 339, 340, 342, 413, 437]',
 '5': '[9, 57, 60, 62, 65, 101, 103, 107, 118, 219, 224, 253, 443, 519, 533, 538, 638, 640, 641, 653, 668, 672, 675, 678, 717, 830, 992, 1018, 1037, 1169, 1260, 1314, 1469, 1713]',
 '6': '[198]',
 '7': '[1, 14, 19, 160, 261, 358, 393, 400, 402, 415, 416, 462, 471, 478, 481, 482, 486, 491, 501, 507]',
 '8': '[29, 42, 44, 58, 59, 77, 93, 230, 246, 247, 255, 357, 366, 372, 387, 610, 611, 613, 670, 879, 881, 882, 883, 895, 896, 911, 920, 972, 1025, 1037, 1038, 1040, 1133, 1141, 1176, 1206, 1207, 1208, 1220, 1233, 1234, 1236, 1240, 1241, 1251, 1253, 1259, 1262, 1263, 1349, 1365, 1366, 1373, 1752, 1941, 2198, 2210, 2284, 2285, 2372, 2373, 2377, 

In [144]:
# Imposible acceder, necesitas claves de elector de cada estado y seccion electoral -- voy a guardar las casillas que no se pudieron

with open('casillas_pendientes.json', 'w') as file:
    json.dump(clean_pendientes, file)


### Casillas Especiales

In [123]:
specials_df = get_casillas_especiales()

In [124]:
specials_df.to_csv("casillas_especiales.csv", encoding="utf-8-sig",
                    index=False)